# A survey on datasets for fairness-aware machine learning

[Paper](https://wires.onlinelibrary.wiley.com/doi/10.1002/widm.1452)

[Slides](https://danjacobellis.github.io/FTML/survey_of_datasets.slides.html)


<script>
    document.querySelector('head').innerHTML += '<style>.slides { zoom: 1.75 !important; }</style>';
</script>

<center> <h1>
A survey on datasets for fairness-aware machine learning
</h1> </center>

## Three ways to intervene in the name of fairness

1. Use a different learning algorithm
2. Post processing of the model outputs
3. Interventions in the original data

* There are many ways that a dataset can lead to an unfair model during ERM
* We need a collection datasets to evaluate potential fairness interventions

## Background and caveats

* We will only explore tabular data
* We will use a **Bayesian network (BN)** to explore the relationships between features
* All numerical features will be discretized to make them categorical
  * Most BN algorithms cannot efficiently handle numeric features
* We will examine relationships between specific, categorical features. Examples:
    * $A_1 = \text{sex} \in \{M,F\}$
    * $A_2 = \text{race} \in \{\text{white},\text{nonwhite}\}$
    * $A_3 = \text{race} \in \{\text{white}, \text{black}, \text{asian-pac-islander}, \text{amer-indian-eskimo}, \text{other}\}$
    * $A_4 = \text{age} \in \{ 17, 18, \dots, 90 \}$
    * $A_5 = \text{age} \in \{ 0, \dots, 255 \}$
    * $A_6 = \text{age} \in \{ 25-60, <25, >60 \}$

## Bayesian Network

[put a simple example]

## Show some of the networks (Ricci)

## Employment 

## Finance

## Criminology

## Healthcare

## Education

## Review of fairness metrics

## Review data balance

## Methodology

* Make all protected classes binary
* Perform logistic regression

## Results showing accuracy and fairness measures

## How can a model discrimintate against both classes simultaneously?